In [3]:
import pandas as pd
from tqdm import tqdm
import pymorphy3
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
train = pd.read_csv(r'C:\Users\User\Desktop\AI_DL_2025\practices\31.03-02.04\train.csv')
train.head()

,title,description,real_salary
0,Программист 1С,Высшее специализированное образование. Опыт ра...,78300.0
1,Инженер-программист,Знание принципов работы RTOS и опыт их использ...,150000.0
2,"Монтажер программы ""Beautytime""",Наличие опыта работы в программе Adobe Premier...,27500.0
3,Программист C#,Высшее техническое образование. Опыт работы: о...,225000.0
4,Продюсер образовательных программ,Уверенный пользователь ПК. Умение работать с т...,30450.0


Набор данных о вакансиях. Будем пытаться прогнозировать зарплату по описанию вакансии

In [ ]:
## 1 выполните предварительную обработку данных (нижний регистр, спецсимволы, лемматизация)

In [5]:
tqdm.pandas()
morph=pymorphy3.MorphAnalyzer()
def process_text(text):
    s = re.sub(r'[^\w\s]+|[\d]+', r'',text).strip()
    s = s.lower()
    s1 = word_tokenize(s) #токенизация
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    return sentence

train['process_text']=train.description.progress_apply(process_text)

100%|██████████| 2100/2100 [00:44<00:00, 46.90it/s]


In [ ]:
## 2 проведите векторизацию ваших данных, используя CountVectorizer или TfIdfVectorizer

In [6]:
russian_stopwords = stopwords.words("russian") #стоп-слова
vectorizer = CountVectorizer(max_features=500, min_df=20, max_df=0.7, stop_words=russian_stopwords)
text_cv = vectorizer.fit_transform(train['process_text'])
text_cv = pd.DataFrame(text_cv.toarray(),columns=vectorizer.get_feature_names_out())
text_cv.head()

,ajax,api,aspnet,backend,bitrix,bootstrap,cms,crm,css,delphi,...,читать,чпу,чтение,чужой,экономический,эксплуатация,электронный,это,являться,язык
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
## 3 разбейте данные на обучающую и тестовую выборки

In [7]:
X = text_cv
Y = train['real_salary']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=123)

In [ ]:
## 4 постройте любую регрессионную модель, используя любые известные вам алгоритмы и оцените её качество на тестовой выборке (метрику выберите сами)

In [9]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()  # объявляем модель
model.fit(X_train, Y_train)  # обучаем

LinearRegression()

In [12]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)
print('MAE:',mean_absolute_error(y_pred,Y_test))

MAE: 52603.87166525441


In [ ]:
# для тестового набора данных с помощью вашей модели предскажите значение real_salary, метрика для проверки MAE

In [ ]:
test_features = pd.read_csv('test_features.csv')
test_features.head()

,title,description
0,Ведущий программист-аналитик 1С,Основные требования к опыту и знаниям: Опыт ра...
1,Инженер-программист (ЧПУ),Образование высшее профессиональное - Технолог...
2,Программист ПЛК (PLC),ПЛК Coolmay управляется через GX Work2 и нам н...
3,Инженер-Программист микроконтроллеров на С/С++...,Хорошее знание С/С++. Опыт разработки под семе...
4,Тестировщик программного обеспечения со знание...,Важно: практический опыт администрирования Lin...


In [ ]:
#пример обученного векторизатора для тестовых данных (не забудьте нормализовать текст в тестовой выборке!)
text_cv_test=vectorizer_cv.transform(text)
text_cv_test = pd.DataFrame(text_cv_test.toarray(),columns=vectorizer_cv.get_feature_names_out())
text_cv_test.head()

,ajax,api,aspnet,backend,bitrix,bootstrap,cms,crm,css,delphi,...,чпу,чтение,чужой,экономический,эксплуатация,электронный,этап,это,являться,язык
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
y_pred = model.predict(text_cv_test)

In [ ]:
#пример формирования датафрейма для загрузки
df = pd.DataFrame()
df['result']=y_pred
df.to_csv('title.csv',index=False)

In [ ]:
# результат прогноза для тестовой выборки просылаем в формате csv,
# где лежит только один столбец result, название файла - Фамилия_Группа.csv
# группа ИБ или БПМ - пример Бимбумбамов_ИБ.csv или Бимбумбамов_БПМ.csv